# Preproccessing script for CMIP6 potential temperature, salinity, and DIC fields

All preprocessing is to be done on the *native* CMIP6 output fields, then included when calculating depth-integrated temperature, salinity, and DIC

Our preprocessing includes the following:
1. Calculation of annual averages using momlevel (takes into account irregular month lengths) -- saving this to a zarr directory
2. Calculation of linear drifts from preindustrial control (piControl) runs. The use of a linear drift rather than a higher-order drift is in line with how XMIP and pangeo operate (double-check this)
3. Regridding CMIP fields to a standard 1x1 (or 1.25x1.25) grid (1x1 for theoretical work, 1.25x1.25 for work with Argo data)
3. Integrating CMIP fields to various depths (100m, 500m, 1000m, 2000m)
4. Concatenating the integrated CMIP fields into a super-timeseries

In [1]:
import xarray as xr

In [2]:
from dask.distributed import Client
client = Client('tcp://127.0.0.1:8786')
client

<Client: 'tcp://192.168.0.4:8786' processes=4 threads=8, memory=14.90 GiB>

In [3]:
var = "dissic"
model = "ACCESS"

In [4]:
#ds = xr.open_mfdataset(f"ann_{var}_{model}*", use_cftime=True, chunks={"i":60, "j":66, "lev":15})
ds = xr.open_zarr(f"ann_{var}_{model}", use_cftime=True)
ds

<xarray.Dataset>
Dimensions:        (time: 200, lev: 50, j: 300, i: 360)
Coordinates:
  * i              (i) int32 0 1 2 3 4 5 6 7 ... 352 353 354 355 356 357 358 359
  * j              (j) int32 0 1 2 3 4 5 6 7 ... 292 293 294 295 296 297 298 299
    latitude       (j, i) float64 dask.array<chunksize=(50, 360), meta=np.ndarray>
  * lev            (lev) float64 5.0 15.0 25.0 ... 5.166e+03 5.499e+03 5.831e+03
    longitude      (j, i) float64 dask.array<chunksize=(50, 360), meta=np.ndarray>
  * time           (time) object 0101-07-02 12:00:00 ... 0300-07-02 12:00:00
Data variables:
    dissic_annual  (time, lev, j, i) float32 dask.array<chunksize=(20, 10, 50, 360), meta=np.ndarray>

In [6]:
var_in = ds.dissic_annual.chunk({"time":200, "lev":10, "j":50, "i":60})
p = var_in.polyfit(dim="time", deg=1)
fit = xr.polyval(ds.time, p.polyfit_coefficients)
drift_100yr = (fit[100,:,:,:] - fit[0,:,:,:]).chunk({"lev":1})

In [7]:
ds_out = drift_100yr.to_dataset(name="drift_100yr")
ds_out

<xarray.Dataset>
Dimensions:      (lev: 50, j: 300, i: 360)
Coordinates:
  * lev          (lev) float64 5.0 15.0 25.0 ... 5.166e+03 5.499e+03 5.831e+03
  * j            (j) int64 0 1 2 3 4 5 6 7 8 ... 292 293 294 295 296 297 298 299
  * i            (i) int64 0 1 2 3 4 5 6 7 8 ... 352 353 354 355 356 357 358 359
Data variables:
    drift_100yr  (lev, j, i) float64 dask.array<chunksize=(1, 300, 360), meta=np.ndarray>

In [8]:
# need to add level bounds for vertical integration -- &*(%£^@&%$@%@*
# pull out bnds coordinate and lev_bnds (or olevel_bnds if you are IPSL)
ds_lvbnd = xr.open_mfdataset(f"/Volumes/KT-TOSHIBA/ENOI/CMIP6/preindustrial_controls/{var}_Omon_{model}*.nc", 
                       use_cftime=True)
ds_lvbnd

<xarray.Dataset>
Dimensions:             (time: 2400, bnds: 2, lev: 50, j: 300, i: 360,
                         vertices: 4)
Coordinates:
  * time                (time) object 0101-01-16 12:00:00 ... 0300-12-16 12:0...
  * lev                 (lev) float64 5.0 15.0 25.0 ... 5.499e+03 5.831e+03
  * j                   (j) int32 0 1 2 3 4 5 6 ... 293 294 295 296 297 298 299
  * i                   (i) int32 0 1 2 3 4 5 6 ... 353 354 355 356 357 358 359
    latitude            (j, i) float64 dask.array<chunksize=(300, 360), meta=np.ndarray>
    longitude           (j, i) float64 dask.array<chunksize=(300, 360), meta=np.ndarray>
Dimensions without coordinates: bnds, vertices
Data variables:
    time_bnds           (time, bnds) object dask.array<chunksize=(120, 2), meta=np.ndarray>
    lev_bnds            (time, lev, bnds) float64 dask.array<chunksize=(120, 50, 2), meta=np.ndarray>
    vertices_latitude   (time, j, i, vertices) float64 dask.array<chunksize=(120, 300, 360, 4), meta=np.ndarray>
    vertices_longitude  (time, j, i, vertices) float64 dask.array<chunksize=(120, 300, 360, 4), meta=np.ndarray>
    dissic              (time, lev, j, i) float32 dask.array<chunksize=(120, 50, 300, 360), meta=np.ndarray>
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  36524.0
    creation_date:          2019-11-13T00:34:47Z
    ...                     ...
    variable_id:            dissic
    variant_label:          r1i1p1f1
    version:                v20191112
    cmor_version:           3.4.0
    tracking_id:            hdl:21.14100/7c275008-2414-49b5-851b-b4e0465c8426
    license:                CMIP6 model data produced by CSIRO is licensed un...

In [9]:
ds_out = ds_out.assign(lev_bnds=ds_lvbnd["lev_bnds"][0,:,:]).drop_vars("time")
ds_out

<xarray.Dataset>
Dimensions:      (lev: 50, j: 300, i: 360, bnds: 2)
Coordinates:
  * lev          (lev) float64 5.0 15.0 25.0 ... 5.166e+03 5.499e+03 5.831e+03
  * j            (j) int64 0 1 2 3 4 5 6 7 8 ... 292 293 294 295 296 297 298 299
  * i            (i) int64 0 1 2 3 4 5 6 7 8 ... 352 353 354 355 356 357 358 359
Dimensions without coordinates: bnds
Data variables:
    drift_100yr  (lev, j, i) float64 dask.array<chunksize=(1, 300, 360), meta=np.ndarray>
    lev_bnds     (lev, bnds) float64 dask.array<chunksize=(50, 2), meta=np.ndarray>

In [10]:
if model == "IPSL":
    ds_out = ds_out.rename({'olevel': 'lev'})
    ds_out = ds_out.assign_coords(lat=(('y', 'x'), ds_lvbnd.nav_lat.data),
                                  lon=(('y', 'x'), ds_lvbnd.nav_lon.data))
elif (model == "MPI") | (model == "ACCESS") | (model == "UKESM") | (model == "CanESM") | (model == "NorESM"):
    ds_out = ds_out.assign_coords(lat=(('j', 'i'), ds.latitude.data), 
                                  lon=(('j', 'i'), ds.longitude.data))
    
ds_out

<xarray.Dataset>
Dimensions:      (lev: 50, j: 300, i: 360, bnds: 2)
Coordinates:
  * lev          (lev) float64 5.0 15.0 25.0 ... 5.166e+03 5.499e+03 5.831e+03
  * j            (j) int64 0 1 2 3 4 5 6 7 8 ... 292 293 294 295 296 297 298 299
  * i            (i) int64 0 1 2 3 4 5 6 7 8 ... 352 353 354 355 356 357 358 359
    lat          (j, i) float64 dask.array<chunksize=(50, 360), meta=np.ndarray>
    lon          (j, i) float64 dask.array<chunksize=(50, 360), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    drift_100yr  (lev, j, i) float64 dask.array<chunksize=(1, 300, 360), meta=np.ndarray>
    lev_bnds     (lev, bnds) float64 dask.array<chunksize=(50, 2), meta=np.ndarray>

In [11]:
%%time
ds_out.to_zarr(f"drift_100yr_{var}_{model}")

CPU times: user 1.08 s, sys: 188 ms, total: 1.27 s
Wall time: 29min 36s


In [ ]:
ds_out.drift_100yr[15,:,:].plot()

In [ ]:
ds

In [ ]:
import matplotlib.pyplot as plt
ds1 = xr.open_zarr("drift_100yr_so_ACCESS")
ds2 = xr.open_zarr("drift_100yr_so_ACCESS2")


fig, axs = plt.subplots(1,2)
ds1.drift_100yr[10,:,:].plot(ax=axs[0])
ds2.drift_100yr[10,:,:].plot(ax=axs[1])

In [ ]:
ds1 = xr.open_zarr("drift_100yr_thetao_ACCESS")
ds2 = xr.open_zarr("drift_100yr_thetao_ACCESS2")


fig, axs = plt.subplots(1,2)
ds1.drift_100yr[10,:,:].plot(ax=axs[0])
ds2.drift_100yr[10,:,:].plot(ax=axs[1])